In [7]:
%pip install nltk
%pip install scikit-learn
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import nltk
import sklearn
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#get the data from the kaggledata_archive for n-grams (using all-data.csv)
financial_csv = pd.read_csv('./kaggledata_archive/all-data.csv', encoding='latin-1')
#Looking at basic info
print(financial_csv.head())
print(financial_csv.shape)


(4845, 2)

In [ ]:
#Task 1: N-gram Generation
from nltk.tokenize import word_tokenize
import pandas as pd
#Task 1: N-gram Generation
def n_gram_generation():
    
    return

unigram_model = n_gram_generation()
bigram_model = n_gram_generation()
trigram_model = n_gram_generation()
#This is the plotting of n-gram models
#plt.title()
#plt.xlabel()
#plt.ylabel()
#plt.legend()
#plt.show()

_IncompleteInputError: incomplete input (710812364.py, line 8)

In [ ]:
#Task 2: N-Gram Modeling

In [ ]:
#Task 3: Handling unseen n-grams

In [ ]:
#Task 4: Interpolation